In [ ]:
# Created 05/13/19 by A.T. (tropiano.4@osu.edu)

# This notebook makes a figure for a highlight for the NUCLEI 2019 collaboration meeting.

from os import chdir, getcwd
from matplotlib.offsetbox import AnchoredText
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.linalg import expm
# Scripts made by A.T.
from deuteron import Deuteron
from Figures import figures_functions as ff
from Potentials.vsrg_macos import load_save_potentials as lp
from SRG_codes.srg_unitary_transformation import SRG_unitary_transformation

__Function for plotting momentum projection operators evolving in $\lambda$__

In [ ]:
def momentum_projection_operator(q, kvnn, channel, kmax, kmid, ntot, method, generator, lambda_bd=0.00, \
                                 k_magnus=6, ds=1e-5):
    '''Plots a 2 x 2 figure of the momentum projection operator evolving from 10.0 to 1.2 fm^-1.'''
    
    # Arguments
    
    # q (float): Momentum value in fm^-1
    # kvnn (integer): This number specifies the potential
    # channel (string): The partial wave channel ('1S0', '3S1', etc.)
    # kmax (float): Maximum value in the momentum mesh
    # kmid (float): Mid-point value in the momentum mesh
    # ntot (integer): Number of momentum points in mesh
    # method (string): The evolution method 'srg' or 'magnus'
    # generator (string): SRG generator ('Wegner', 'T', 'Block-diag')
    # lambda_bd (float): Lambda value for block-diagonal decoupling (e.g. 2.00 fm^-1)
    # k_magnus (integer): Number of terms to include in Magnus sum (for Magnus only)
    # ds (float): Step-size in the flow parameter s (for Magnus only)
    
    # Load initial Hamiltonian, momentum, and weights
    H0_matrix = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
    k_array, k_weights = lp.load_momentum(kvnn, channel, kmax, kmid, ntot)
    
    # Pick the nearest value of q if q isn't exactly in k_array
    if not( q in k_array ):
        
        k_difference_array = abs(k_array-q)
        k_difference_min = min(k_diff_array)
        q_index = list(k_difference_array).index(k_difference_min)
        # Reset q to an actual value in k_array mesh (this is the closest value to entered q)
        q = k_array[q_index]    
    
    # Initialize deuteron class 
    D = Deuteron(H_matrix, k_array, k_weights)
    
    # Loop over lambda values in lambdas and save to dictionary where key = lambda
    lambdas = [10.0, 2.8, 2.0, 1.2]
    d = {}
    
    for lamb in lambdas:
        
        # Load unitary transformation
        # SRG calls function which builds U(s) out of un-evolved and evolved eigenvectors
        if method == 'srg':
            
            # Must input evolved matrix
            Hs_matrix = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, method, generator, lamb, lambda_bd)
            Us_matrix = SRG_unitary_transformation(H0_matrix, Hs_matrix)
            
        # Magnus loads Omega(s) to compute U(s) = exp^Omega(s)
        elif method == 'magnus':
            
            Os_matrix = lp.load_omega(kvnn, channel, kmax, kmid, ntot, generator, lamb, lambda_bd, k_magnus, ds)
            Us_matrix = expm(Os_matrix)
        
        # Evolved momentum projection operator
        momemtum_proj_operator = D.momentum_proj_operator(q, U=Us_matrix)
        # Enter into dictionary
        d[lamb] = momentum_proj_operator
        
    # Plot the 2 x 2 figure
    
    # ... 
            
        

__Run function in the cell below__